# Voorbeeld van omzetting door Veg2Hab

Dit notebook doorloopt alle stappen die tot nu toe geimplementeerd zijn voor Veg2Hab. Het is een voorbeeld van hoe de omzetting van vegetatie naar habitattypekaart kan verlopen.

Eerst worden alle benodigde resources geinstantieerd (was-wordt lijst, definitietabel, fgr-kaart). Hierna wordt de omzetting stap voor stap uitgevoerd.

In [1]:
from pathlib import Path

import sys
sys.path.append('../')

from veg2hab.waswordtlijst import WasWordtLijst, opschonen_was_wordt_lijst
from veg2hab.definitietabel import DefinitieTabel, opschonen_definitietabel
from veg2hab.vegkartering import Kartering
import pandas as pd
from veg2hab.fgr import FGR

pd.set_option('display.max_columns', 100)

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### Was-Wordt lijst opschonen en inladen

VvN en SBB codes worden gecheckt op validiteit bij:
1. het opschonen van een was-wordt lijst
2. bij het inlezen van een opgeschoonde was-wordt lijst

In [2]:
path_in_wwl = Path("../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx")
path_out_wwl = Path("../testing/opgeschoonde_waswordt.xlsx")
opschonen_was_wordt_lijst(path_in_wwl, path_out_wwl)
wwl = WasWordtLijst.from_excel(path_out_wwl)
wwl.df.head(3)

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide


,VvN,SBB
0,14rg10,14-l
1,None,7-a
2,None,8b-b


### Definitietabel opschonen en inladen

VvN- en SBB-codes worden gecheckt op validiteit bij:
1. het opschonen van een definitietabel
2. bij het inlezen van een opgeschoonde definitietabel

In [3]:
path_in_dt = Path("../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls")
path_in_mitsjson = Path("../data/mitsjson.csv")
path_out_dt = Path("../testing/opgeschoonde_definitietabel.xlsx")
opschonen_definitietabel(path_in_dt, path_in_mitsjson, path_out_dt)
dt = DefinitieTabel.from_excel(path_out_dt)
dt.df.head(3)

Alle SBB codes zijn valide
Alle VvN codes zijn valide


,DT regel,Habitattype,Kwaliteit,SBB,VvN,mits,mozaiek,mitsjson,Criteria
0,2,H1110_A,Kwaliteit.GOED,50a,None,mits in de slikkige en fijnzandige delen van F...,,"{ ""type"": ""EnCriteria"", ""sub_criteria""...",((FGR is Getijdengebied of FGR is Noordzee) en...
1,3,H1110_A,Kwaliteit.GOED,50b,None,mits in de slikkige en fijnzandige delen van F...,,"{ ""type"": ""EnCriteria"", ""sub_criteria""...",((FGR is Getijdengebied of FGR is Noordzee) en...
2,4,H1110_A,Kwaliteit.GOED,50c,None,mits in de slikkige en fijnzandige delen van F...,,"{ ""type"": ""EnCriteria"", ""sub_criteria""...",((FGR is Getijdengebied of FGR is Noordzee) en...


### FGR inladen

Bij het inladen worden de FGR typen omgezet naar de FGRType Enum, en hierdoor vind ook validatie plaats.

In [4]:
fgr = FGR(Path("../data/bronbestanden/FGR.json"))
fgr.gdf.head(3)

,fgr,geometry
0,FGRType.DU,"POLYGON ((105147.000 526863.188, 105092.000 52..."
1,FGRType.DU,"POLYGON ((111314.000 552409.000, 111289.008 55..."
2,FGRType.DU,"POLYGON ((119244.333 577690.288, 119230.924 57..."


## Inladen van een kartering vanaf een shapefile

In [5]:
shp_path = Path("../testing/vegetatiekarteringen/GR/SGL Hunzedal en Leekstermeer2021/2021 Vegetatiekartering Leekstermeer2021/GIS bestanden Onlanden 2021/Vegetatiekartering_Leekstermeer2021.shp")
Kartering.from_shapefile(shp_path, "elmid", vegtype_col_format="single", sbb_of_vvn="sbb", SBB_col="SBBTYPE", split_char="+").gdf

,elmid,geometry,SBBTYPE,SBBTYPE1,SBBTYPE2,SBBTYPE3,Datum,Opmerking,Opp,_LokVrtNar,perc_0,perc_1,perc_2,VegTypeInfo
0,144,"POLYGON ((225536.382 578550.118, 225536.613 57...",08C-a+16-a,8c-a,16-a,None,None,None,1692.139818,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['8c-a'], VvN: []), (50.0%, SBB:..."
1,143,"POLYGON ((225518.346 578389.608, 225514.596 57...",08-a+08C2c,8-a,8c2c,None,None,None,2974.448552,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['8-a'], VvN: []), (50.0%, SBB: ..."
2,73,"POLYGON ((225398.991 578372.192, 225399.792 57...",16-a,16-a,None,None,None,None,4582.870987,Lokale typologie is primair vertaald naar sbb,100.0,0.0,0.0,"[(100.0%, SBB: ['16-a'], VvN: [])]"
3,62,"POLYGON ((225338.120 578506.385, 225330.908 57...",12B1d+50A,12b1d,50a,None,None,None,2843.780077,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['12b1d'], VvN: []), (50.0%, SBB..."
4,111,"POLYGON ((225731.271 578748.007, 225730.075 57...",50A+08B3d,50a,8b3d,None,None,None,1911.539469,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['50a'], VvN: []), (50.0%, SBB: ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,193,"POLYGON ((225335.414 578905.115, 225334.448 57...",12B1d,12b1d,None,None,None,None,442.032701,Lokale typologie is primair vertaald naar sbb,100.0,0.0,0.0,"[(100.0%, SBB: ['12b1d'], VvN: [])]"
267,220,"POLYGON ((225121.987 578757.930, 225125.003 57...",08-k,8-k,None,None,None,None,361.533352,Lokale typologie is primair vertaald naar sbb,100.0,0.0,0.0,"[(100.0%, SBB: ['8-k'], VvN: [])]"
268,58,"POLYGON ((225297.768 578230.760, 225292.476 57...",08B3a+08C-f,8b3a,8c-f,None,None,None,722.439797,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['8b3a'], VvN: []), (50.0%, SBB:..."
269,249,"POLYGON ((224515.275 578618.372, 224510.014 57...",39A1c+08B3a,39a1c,8b3a,None,None,None,713.229564,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['39a1c'], VvN: []), (50.0%, SBB..."


## Inladen en omzetten van een access kartering

- Voor ieder vlak in de kartering wordt de (complexe) SBB code opgezocht.
- Deze SBB-codes worden opgeschoond en gevalideerd.
- De SBB codes worden toegevoegd aan de dataframe van de kartering als VegTypeInfo instance.
  
De resulterende dataframe heeft voor elke regel in KarteringVegetatietype.csv een VegTypeInfo instance in de rij van de betreffende shape.

In [6]:
shape_path = Path("../data/notebook_data/Rottige_Meenthe_Brandemeer_2013/vlakken.shp")
csvs_path = Path("../data/notebook_data/Rottige_Meenthe_Brandemeer_2013/864_RottigeMeenthe2013/")
shape_elm_id_column = "ElmID"

access_kartering = Kartering.from_access_db(shape_path, shape_elm_id_column, csvs_path)

access_kartering.gdf.head(3)


,ElmID,Opmerking,Datum,geometry,Opp,_LokVrtNar,intern_id,Locatie,VegTypeInfo
0,28049,None,19072013,"POLYGON ((189433.818 538314.302, 189439.215 53...",3761.736281,Lokale typologie is primair vertaald naar SBB,1809,1809,"[(90%, SBB: ['9a2a'], VvN: []), (10%, SBB: ['9..."
1,15034,None,10092013,"POLYGON ((190568.500 539171.122, 190538.023 53...",10442.658431,Lokale typologie is primair vertaald naar SBB,815,815,"[(100%, SBB: ['50a'], VvN: [])]"
2,13503,None,14102013,"POLYGON ((189538.104 539409.332, 189532.399 53...",2174.098680,Lokale typologie is primair vertaald naar SBB,650,650,"[(90%, SBB: ['9a2a'], VvN: []), (10%, SBB: ['9..."


#### Opzoeken en toevoegen VvN aan kartering

In [7]:
access_kartering.apply_wwl(wwl)
# Voorbeeld van 3 vegtypeinfos in 1 geometry
access_kartering.gdf["VegTypeInfo"].iloc[3]

[VegTypeInfo(percentage=80, SBB=[SBB(klasse='9', verbond='b', associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='a')], VvN=[]),
 VegTypeInfo(percentage=10, SBB=[SBB(klasse='9', verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='g')], VvN=[VvN(klasse='9', orde=None, verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='3')]),
 VegTypeInfo(percentage=10, SBB=[SBB(klasse='9', verbond='a', associatie='2', subassociatie='b', derivaatgemeenschap=None, rompgemeenschap=None)], VvN=[VvN(klasse='9', orde='a', verbond='a', associatie='2', subassociatie='b', derivaatgemeenschap=None, rompgemeenschap=None)])]

#### Vinden van de mogelijke habitattypen voor een VegTypeInfo

Voor elke VegTypeInfo worden de mogelijke habitattypen opgezocht in de definitietabel. Deze worden vervolgens in een HabitatVoorstel gezet, samen met de code waarop de match is gemaakt, de regel in de definitietabel die is gebruikt en het niveau van de match.

In [8]:
access_kartering.apply_deftabel(dt)
access_kartering.gdf["HabitatVoorstel"]

0       [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1       [[HabitatVoorstel(onderbouwend_vegtype=SBB(kla...
2       [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
3       [[HabitatVoorstel(onderbouwend_vegtype=SBB(kla...
4       [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
                              ...                        
1944    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1945    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1946    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1947    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1948    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
Name: HabitatVoorstel, Length: 1949, dtype: object

In [9]:
# Voorbeeld met voor 2 VegTypeInfo's waarbij de eerste twee mogelijke voorstellen heeft
access_kartering.gdf["HabitatVoorstel"][45]

[[HabitatVoorstel(onderbouwend_vegtype=VvN(klasse='40', orde='a', verbond='a', associatie='2', subassociatie='a', derivaatgemeenschap=None, rompgemeenschap=None), vegtype_in_dt=VvN(klasse='40', orde='a', verbond='a', associatie='2', subassociatie=None, derivaatgemeenschap=None, rompgemeenschap=None), habtype='H2180_B', kwaliteit=<Kwaliteit.GOED: 'Goed'>, idx_in_dt=178, mits=FGRCriterium(fgrtype=<FGRType.DU: 'Duinen'>), mozaiek=GeenMozaiekregel(), match_level=6),
  HabitatVoorstel(onderbouwend_vegtype=VvN(klasse='40', orde='a', verbond='a', associatie='2', subassociatie='a', derivaatgemeenschap=None, rompgemeenschap=None), vegtype_in_dt=VvN(klasse='40', orde='a', verbond='a', associatie='2', subassociatie=None, derivaatgemeenschap=None, rompgemeenschap=None), habtype='H91D0', kwaliteit=<Kwaliteit.GOED: 'Goed'>, idx_in_dt=650, mits=EnCriteria(sub_criteria=[NietCriterium(sub_criterium=FGRCriterium(fgrtype=<FGRType.DU: 'Duinen'>)), PlaceholderCriterium()]), mozaiek=GeenMozaiekregel(), matc

### De mitsen checken en de HabitatVoorstellen omzetten naar HabitatKeuzes

In [10]:
access_kartering.check_mitsen(fgr)
print(access_kartering.gdf["HabitatKeuze"][5][1])
print(access_kartering.gdf["VegTypeInfo"][5][1])

HabitatKeuze(status=<KeuzeStatus.MEERDERE_KLOPPENDE_MITSEN: 5>, habtype='HXXXX', kwaliteit=<Kwaliteit.ONBEKEND: 'Onbekend'>, opmerking="Er zijn meerdere habitatvoorstellen die aan hun mitsen voldoen; Kloppende mitsen: [['11ba2', 'H4010_B', 'FGR is Laagveengebied'], ['11ba2a', 'H4010_B', 'FGR is Laagveengebied'], ['11ba2b', 'H4010_B', 'FGR is Laagveengebied']]", debug_info='', habitatvoorstellen=[HabitatVoorstel(onderbouwend_vegtype=VvN(klasse='11', orde='b', verbond='a', associatie='2', subassociatie=None, derivaatgemeenschap=None, rompgemeenschap=None), vegtype_in_dt=VvN(klasse='11', orde='b', verbond='a', associatie='2', subassociatie=None, derivaatgemeenschap=None, rompgemeenschap=None), habtype='H4010_B', kwaliteit=<Kwaliteit.GOED: 'Goed'>, idx_in_dt=386, mits=FGRCriterium(fgrtype=<FGRType.LV: 'Laagveengebied'>), mozaiek=GeenMozaiekregel(), match_level=6), HabitatVoorstel(onderbouwend_vegtype=VvN(klasse='11', orde='b', verbond='a', associatie='2', subassociatie='a', derivaatgemeens

In [34]:
from veg2hab.criteria import GeenCriterium

# > access_kartering.gdf.HabitatVoorstel.iloc[0]
# [[HabitatVoorstel(onderbouwend_vegtype=VvN(klasse='9', orde='a', verbond='a', associatie='2', subassociatie='a', derivaatgemeenschap=None, rompgemeenschap=None), vegtype_in_dt=VvN(klasse='9', orde='a', verbond='a', associatie='2', subassociatie=None, derivaatgemeenschap=None, rompgemeenschap=None), habtype='H7140_B', kwaliteit=<Kwaliteit.GOED: 'Goed'>, idx_in_dt=566, mits=GeenCriterium(), mozaiek=GeenMozaiekregel(), match_level=6)],
#  [HabitatVoorstel(onderbouwend_vegtype=VvN(klasse='9', orde='a', verbond='a', associatie='2', subassociatie='a', derivaatgemeenschap=None, rompgemeenschap=None), vegtype_in_dt=VvN(klasse='9', orde='a', verbond='a', associatie='2', subassociatie=None, derivaatgemeenschap=None, rompgemeenschap=None), habtype='H7140_B', kwaliteit=<Kwaliteit.GOED: 'Goed'>, idx_in_dt=566, mits=GeenCriterium(), mozaiek=GeenMozaiekregel(), match_level=6)]]

# filter out rows with GeenCriterium()

access_kartering.gdf["mits"] = access_kartering.gdf.HabitatVoorstel.apply(lambda x: [[z.mits for z in y] for y in x])
pd.set_option('display.max_colwidth', 150)

In [35]:
access_kartering.gdf["mits"]

0                                                                                                      [[geen mits (altijd waar)], [geen mits (altijd waar)]]
1       [[((FGR is Getijdengebied of FGR is Noordzee) en placeholder), ((FGR is Getijdengebied of FGR is Noordzee) en placeholder), (FGR is Getijdengebied...
2                                                                                                      [[geen mits (altijd waar)], [geen mits (altijd waar)]]
3                        [[geen mits (altijd waar)], [(FGR is Noordzee of FGR is Getijdengebied of FGR is Duinen of placeholder)], [geen mits (altijd waar)]]
4         [[geen mits (altijd waar)], [placeholder], [(FGR is Noordzee of FGR is Getijdengebied of FGR is Duinen of placeholder)], [geen mits (altijd waar)]]
                                                                                ...                                                                          
1944                [[(FGR is Duinen en placeholder)

In [ ]:
access_kartering.gdf["mits"].iloc[0]


### De kartering formatten als een Habitattypenkartering

Dit pakt de HabitatVoorstellen uit en format het geheel als een Habitattypenkartering zoals beschreven in Gegevens Leverings Protocol (uitvraag bijlage 3a).

Voor nu checken we nog geen mitsen en mozaiek en wordt voor ieder VegTypeInfo het eerste HabitatVoorstel gebruikt.

In [11]:
final_format = access_kartering.as_final_format()
final_format

,Area,Opm,Datum,geometry,_ChkNodig,_Samnvttng,_LokVrtNar,Habtype1,Perc1,Opp1,Kwal1,Opm1,VvN1,SBB1,_Status1,_Uitleg1,_VvNdftbl1,_SBBdftbl1,_VgTypInf1,_ChkNodig1,Habtype2,Perc2,Opp2,Kwal2,Opm2,VvN2,SBB2,_Status2,_Uitleg2,_VvNdftbl2,_SBBdftbl2,_VgTypInf2,_ChkNodig2,Habtype3,Perc3,Opp3,Kwal3,Opm3,VvN3,SBB3,_Status3,_Uitleg3,_VvNdftbl3,_SBBdftbl3,_VgTypInf3,_ChkNodig3,Habtype4,Perc4,Opp4,Kwal4,Opm4,VvN4,SBB4,_Status4,_Uitleg4,_VvNdftbl4,_SBBdftbl4,_VgTypInf4,_ChkNodig4,Habtype5,Perc5,Opp5,Kwal5,Opm5,VvN5,SBB5,_Status5,_Uitleg5,_VvNdftbl5,_SBBdftbl5,_VgTypInf5,_ChkNodig5
0,3761.736281,None,19072013,"POLYGON ((189433.818 538314.302, 189439.215 53...",False,100.0% H7140_B,Lokale typologie is primair vertaald naar SBB,H7140_B,90,338556,G,Er is een duidelijke keuze. Kloppende mits: ge...,9aa2a,None,KeuzeStatus.DUIDELIJK,Als alle regels gevolgd worden is er 1 duideli...,"['9aa2', 566]",None,"(90%, SBB: ['9a2a'], VvN: ['9aa2a'])",False,H7140_B,10,37617.4,G,Er is een duidelijke keuze. Kloppende mits: ge...,9aa2a,None,KeuzeStatus.DUIDELIJK,Als alle regels gevolgd worden is er 1 duideli...,"['9aa2', 566]",None,"(10%, SBB: ['9a2a'], VvN: ['9aa2a'])",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10442.658431,None,10092013,"POLYGON ((190568.500 539171.122, 190538.023 53...",True,100.0% HXXXX,Lokale typologie is primair vertaald naar SBB,HXXXX,100,1044265.8431475804,X,Er zijn habitatvoorstellen met mozaiekregels: ...,"[None, None, None, None, None, None, None, Non...","['50a', '50a', '50a', '50a', '50a', '50a', '50...",KeuzeStatus.WACHTEN_OP_MOZAIEK,Veg2Hab kan nog geen mozaiekregels checken,"[None, None, None, None, None, None, None, Non...","[""['50a', 2]"", ""['50a', 5]"", ""['50a', 12]"", ""[...","(100%, SBB: ['50a'], VvN: [])",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2174.098680,None,14102013,"POLYGON ((189538.104 539409.332, 189532.399 53...",False,100.0% H7140_B,Lokale typologie is primair vertaald naar SBB,H7140_B,90,195669,G,Er is een duidelijke keuze. Kloppende mits: ge...,9aa2a,None,KeuzeStatus.DUIDELIJK,Als alle regels gevolgd worden is er 1 duideli...,"['9aa2', 566]",None,"(90%, SBB: ['9a2a'], VvN: ['9aa2a'])",False,H7140_B,10,21741,G,Er is een duidelijke keuze. Kloppende mits: ge...,9aa2a,None,KeuzeStatus.DUIDELIJK,Als alle regels gevolgd worden is er 1 duideli...,"['9aa2', 566]",None,"(10%, SBB: ['9a2a'], VvN: ['9aa2a'])",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18878.050297,None,12092013,"POLYGON ((190801.313 539561.740, 190819.617 53...",True,"90.0% HXXXX, 10.0% H7140_B",Lokale typologie is primair vertaald naar SBB,HXXXX,80,1.51024e+06,X,Er zijn habitatvoorstellen met mozaiekregels: ...,None,9b-a,KeuzeStatus.WACHTEN_OP_MOZAIEK,Veg2Hab kan nog geen mozaiekregels checken,None,"['9b-a', 342]","(80%, SBB: ['9b-a'], VvN: [])",True,HXXXX,10,188781,X,Er zijn mitsen met nog niet geimplementeerde c...,9rg3,None,KeuzeStatus.PLACEHOLDER_CRITERIA,Er zijn placeholder criteria gevonden; deze ku...,"['9rg3', 230]",None,"(10%, SBB: ['9-g'], VvN: ['9rg3'])",True,H7140_B,10,188781,G,Er is een duidelijke keuze. Kloppende mits: ge...,9aa2b,None,KeuzeStatus.DUIDELIJK,Als alle regels gevolgd worden is er 1 duideli...,"['9aa2', 566]",None,"(10%, SBB: ['9a2b'], VvN: ['9aa2b'])",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4950.611630,None,02102013,"POLYGON ((190497.231 537724.945, 190505.062 53...",True,"61.0% H0000, 39.0% HXXXX",Lokale typologie is primair vertaald naar SBB,HXXXX,29,143568,X,Er zijn mitsen met nog niet geimplementeerde c...,8bd2,None,KeuzeStatus.PLACEHOLDER_CRI

In [12]:
access_kartering.final_format_to_file(Path("../testing/veg2hab_output.shp"))